# 从零开始构建一个大型语言模型

本文是使用PyTorch从0开始逐步实现一个类似ChatGPT那样的大型语言模型（以下简称LLM），步骤大体分为如下：
  - 数据准备与预处理
  - 模型架构与实现
  - 模型训练与评估
  - 文本生成与微调

## 前言

如果你让ChatGPT继续输出“每一次努力都让你感动”后面的内容，ChatGPT会自动延续后续内容并输出，我第一次看到就对此产生了好奇。本文是实现一个类似这样的类GPT模型，且针对文本数据，力求尽量不出现公式，尽量减少专业术语，使用简单清晰的词语进行解释，并给出代码实现。初步可能为了更好的理解，实现一个简单的版本，它可能输出的并不理想，但最终会调整为适合现有大模型的复杂实现思路。故你可以将本文的目的理解为新手入门LLM通识训练。
废话不多说，现在开始！


首先，我们要先明确我们此行的目的：给一段输入“每一次努力都让你感动”，经过模型的处理加工后，输出后续的内容“未来的精彩由此慢慢绽放”。用程序化的语言可以这么表达：输入input，经过model的处理，输出output。一般情况下，模型是依据历史文本来生成固定长度的内容，不可能无限制的生成，所以还需要限定输入长度和输出长度，由此我们定义一个简单的文本生成方法 `generate_text_simple`:

In [4]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    return idx

这个方法就是最终我们需要输出新内容所需要的，我们对这个方法进行解释：
  - `model` : 用来生成文本的大规模语言模型。这个模型经过训练，可以根据给定的输入（上下文）预测并生成接下来最有可能出现的文本。
  - `idx` : 这通常指代的是输入文本序列中每个词或标记（token）在词汇表中的索引位置。在处理过程中，文本首先会被分词器转换成一系列的标记，然后每个标记会根据词汇表映射为一个索引值，用于模型的计算。
  - `max_new_tokens` : 这是设定的一个上限，表示模型在生成新文本时最多可以输出的新标记数量。它有助于控制生成文本的长度，避免生成过长的内容。
  - `context_size` : 模型在做预测时所参考的历史信息长度，即每次提供给模型的输入序列的长度。较大的上下文大小可以让模型记住更多过去的信息，从而可能生成更加连贯和有意义的文本。


为什么要用词汇表索引位置，而不是文本本身？
- 数字化表示：计算机擅长处理数值数据
- 减少复杂度：减少计算复杂度和存储需求
- 迁移学习：可以在另外一套定义好的词汇表上使用预训练的模型
- 支持嵌入层：离散的词汇转换为连续的密集向量，捕捉句子结构和上下文信息
- 泛化能力：学习关系模式，增强适应性

我们试着简单的实现这个方法：
- 首先定义 `model` 参数的类型，名字为 `GPTModel`, 因为我们不知道如何实现它，姑且暂时将输入作为输出返回:

In [50]:
import torch
import torch.nn as nn
print("torch version:",torch.__version__)

class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, in_idx):
        return in_idx


torch version: 2.5.1+cu124


- 然后实现 `generate_text_simple` 方法:

In [167]:
import random
def generate_text_simple(model, idx, max_new_tokens, context_size):

    for _ in range(max_new_tokens):
        idx_cond = idx[-context_size:] # 获取上下文token

        logits = model(idx_cond) # 通过模型生成后续序列
        
        idx_next = random.choices(logits)[0] # 随机选择一个

        idx.append(idx_next) #将下一个预测添加到序列中

    return idx

- 最后我们进行测试，并且有了输出，最大续写5个字，每次只处理上下文4个长度：

In [171]:
model = GPTModel()
in_idx = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
generate_text_simple(model=model, idx=in_idx, max_new_tokens=5, context_size=4)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 13, 16, 16, 16, 16]

但这个结果人类看不懂，并且是很随机的输出，所以它毫无意义。因此我们需要将这个模型进行完善，分析以上过程和代码，我们需要解决的问题有：
- 将输入文本转为词汇表的索引位置
- 实现模型预测，既完成模型训练
- 选择最有可能或最优的预测，而不是随机输出
- 使用正常的文本来测试，且将结果输出为人类能看懂的文本

## 1. 数据准备与预处理

### 1.1 简单分词器

首先，要解决的是将文本转换为数值表示。为此我们需要一份词汇表，现在有很多公开的词汇表可以使用。但为了学习目的，我们从一份中文内容中自定义词汇表，当然这篇中文文章也是后续训练数据集。

- 加载我们要处理的原始文本
- 《每一滴汗水都是未来花朵的养分》是由GPT根据我的提示词生成的文章。

In [17]:
with open("the-road.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("总字数：", len(raw_text))
print(raw_text[:99])

总字数： 778
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己故事中的主角。每一天，我们都面临着选择，每一个选择都是一条通往未知的道路。而在这无数的选择中，有一种选择是永恒不变的——那就是付出努


- 然后我们需要将这篇文章 `token` 化，考虑一些常见的中文标点符号

In [20]:
import re
pattern = r'([\u4e00-\u9fff，。_！？、；：“”‘’()（）——])' # 匹配常见的中文标点符号
result = [token for token in re.split(pattern, raw_text) if token.strip()] # 分割并移除空字符串
print(result)
print(len(result))

['每', '一', '滴', '汗', '水', '都', '是', '未', '来', '花', '朵', '的', '养', '分', '—', '—', '在', '生', '命', '的', '旅', '途', '上', '，', '我', '们', '每', '个', '人', '都', '是', '自', '己', '故', '事', '中', '的', '主', '角', '。', '每', '一', '天', '，', '我', '们', '都', '面', '临', '着', '选', '择', '，', '每', '一', '个', '选', '择', '都', '是', '一', '条', '通', '往', '未', '知', '的', '道', '路', '。', '而', '在', '这', '无', '数', '的', '选', '择', '中', '，', '有', '一', '种', '选', '择', '是', '永', '恒', '不', '变', '的', '—', '—', '那', '就', '是', '付', '出', '努', '力', '。', '每', '一', '次', '的', '努', '力', '，', '无', '论', '大', '小', '，', '都', '在', '悄', '然', '改', '变', '着', '我', '们', '的', '命', '运', '轨', '迹', '。', '它', '们', '累', '积', '起', '来', '，', '成', '为', '我', '们', '成', '长', '道', '路', '上', '最', '宝', '贵', '的', '财', '富', '。', '每', '一', '次', '努', '力', '都', '让', '你', '感', '动', '。', '这', '不', '仅', '仅', '是', '对', '个', '人', '成', '就', '的', '一', '种', '赞', '美', '，', '更', '是', '对', '坚', '持', '和', '毅', '力', '的', '颂', '扬', '。', '当', '我', '们', '看', '到', '运', '动', '员', '在', '赛',

- 从这 `768` 个 `token` 中，我们构建一个包含所有唯一 `token` 的词汇表：

In [53]:
all_tokens = sorted(set(result))
vocab = {token : integer for integer,token  in enumerate(all_tokens)}
vocab_size = len(vocab)
print(vocab_size)

321


打印前后5个元素看一下效果：

In [38]:
for i, item in enumerate(vocab.items()):
    if i <= 5 or i >= (vocab_size-5):
        print(item)
    elif i == 6 or i == (vocab_size-5):
        print('...')

(',', 0)
('—', 1)
('“', 2)
('”', 3)
('、', 4)
('。', 5)
...
('验', 316)
('默', 317)
('鼓', 318)
('，', 319)
('；', 320)


- 接下来我们要用这个词汇表将文本进行 `token` 化，可以理解成进行 `编码`, 当然后续也需要将这些 `token` ID 转化回文本形式，即进行 `解码`，所以我们将实现这两个方法，并放到一个 `SimpleTokenizer` 类中。其中，`encode` 实现将文本转换为  `token` ID, `decode` 实现将 `token` ID 重新转换为文本：

In [50]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        pattern = r'([\u4e00-\u9fff，。_！？、；：“”‘’()（）——])' # 匹配常见的中文标点符号
        preprocessed = [item.strip() for item in re.split(pattern, text) if item.strip()] # 分割并移除空字符串
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = "".join([self.int_to_str[i] for i in ids])
        return text

我们可以使用 `SimpleTokenizer` 将文本编码为数值，然后可以将数值嵌入作为LLM的输入：

In [51]:
tokenizer = SimpleTokenizer(vocab=vocab)

ids = tokenizer.encode("每一次努力都让你感动")
print(ids)

[199, 6, 194, 55, 50, 298, 264, 38, 142, 53]


我们还可以将这些数值解码回文本：

In [52]:
tokenizer.decode(ids)

'每一次努力都让你感动'

- 当然这个分词器还不完整，例如对于未知字没有特殊的处理，会导致代码运行错误，为此，我们可以添加一些特殊的标记：
    - `"<|unk|>"` 表示未知单词
    - `"<|endoftext|>"` 表示文本的结尾

In [61]:
all_tokens = sorted(set(result))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(vocab.items()))
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

323
('鼓', 318)
('，', 319)
('；', 320)
('<|endoftext|>', 321)
('<|unk|>', 322)


同时需要修改 `SimpleTokenizer`，以便知道何时以及如何使用新的`<unk>` 标记

In [72]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        pattern = r'([\u4e00-\u9fff，。_！？、；：“”‘’()（）——])' # 匹配常见的中文标点符号
        preprocessed = [item.strip() for item in re.split(pattern, text) if item.strip()] # 分割并移除空字符串
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = "".join([self.int_to_str[i] for i in ids])
        return text

- **这个分词器有没有什么问题？** 
  - **当然，因为大部分情况下需要保留词语或短语，而不只仅仅单个字，它不会理解语义或语法结构，只是机械式的切割** 
  - **这里我们暂时以这个为主，后续来进行完善，假设我们分词器已完成，来完成剩余部分。**  

让我们尝试使用修改后的标记器来标记文本：

In [73]:
tokenizer = SimpleTokenizer(vocab=vocab)

ids = tokenizer.encode("每一次努力都让你感动啊")
print(ids)

[199, 6, 194, 55, 50, 298, 264, 38, 142, 53, 322]


In [74]:
tokenizer.decode(ids)

'每一次努力都让你感动<|unk|>'

对整个原始文本进行编码：

In [79]:
ids = tokenizer.encode(raw_text + "<|endoftext|>")
print(ids)

[199, 6, 210, 204, 201, 298, 176, 184, 188, 256, 185, 223, 41, 45, 1, 1, 87, 219, 80, 223, 170, 293, 7, 319, 145, 32, 199, 12, 25, 298, 176, 255, 121, 165, 22, 13, 223, 16, 262, 5, 199, 6, 99, 319, 145, 32, 298, 313, 14, 228, 292, 154, 319, 199, 6, 12, 292, 154, 298, 176, 6, 187, 294, 130, 184, 229, 223, 296, 277, 5, 250, 87, 287, 171, 168, 223, 292, 154, 13, 319, 181, 6, 234, 292, 154, 176, 202, 137, 9, 68, 223, 1, 1, 297, 118, 176, 29, 44, 55, 50, 5, 199, 6, 194, 223, 55, 50, 319, 171, 266, 98, 116, 319, 298, 87, 139, 214, 163, 68, 228, 145, 32, 223, 80, 286, 280, 289, 5, 106, 32, 241, 236, 275, 188, 319, 144, 15, 145, 32, 144, 304, 296, 277, 7, 180, 108, 272, 223, 270, 112, 5, 199, 6, 194, 55, 50, 298, 264, 38, 142, 53, 5, 287, 9, 26, 26, 176, 113, 12, 25, 144, 118, 223, 6, 234, 274, 247, 319, 179, 176, 113, 90, 155, 81, 198, 50, 223, 314, 150, 5, 128, 145, 32, 227, 47, 286, 53, 79, 87, 273, 89, 7, 157, 205, 204, 201, 319, 235, 105, 111, 87, 109, 316, 110, 300, 172, 96, 303, 231, 31

In [80]:
tokenizer.decode(ids)

'每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己故事中的主角。每一天，我们都面临着选择，每一个选择都是一条通往未知的道路。而在这无数的选择中，有一种选择是永恒不变的——那就是付出努力。每一次的努力，无论大小，都在悄然改变着我们的命运轨迹。它们累积起来，成为我们成长道路上最宝贵的财富。每一次努力都让你感动。这不仅仅是对个人成就的一种赞美，更是对坚持和毅力的颂扬。当我们看到运动员在赛场上挥洒汗水，科学家在实验室里日夜钻研，或是普通劳动者在岗位上默默耕耘时，我们内心深处都会被触动。这种感动源于对人类潜能的敬佩，以及对那些为了梦想而不惜一切代价的人们的尊重。未来的精彩由此慢慢绽放。就像一朵花从种子开始，在土壤中吸收养分，经过长时间的孕育，最终破土而出，展现出它的美丽一样。我们的未来也是这样，通过不断的努力，我们将积累起足够的力量，去迎接即将到来的挑战与机遇。每一次小小的进步，每一点微不足道的成功，都是通向更大辉煌的基石。在这个过程中，我们需要明白，成功并非一蹴而就。它是由无数次失败、反思和再尝试构成的。因此，当我们在追求梦想的路上遇到困难时，不要轻易放弃。相反，应该把每一次挫折看作是学习的机会，从中吸取教训，并且更加坚定地朝着目标前进。同时，我们也应当珍惜身边那些支持和鼓励我们的人。他们可能是家人、朋友、老师或者是陌生人。正是有了他们的陪伴和支持，我们才能更有勇气面对生活中的风风雨雨。所以，让我们心怀感恩之心，用实际行动回报那些帮助过我们的人。总之，“每一次努力都让你感动,未来的精彩由此慢慢绽放”这句话不仅仅是一句简单的口号，它蕴含着深刻的人生哲理。它提醒我们要珍惜每一个当下，用心去感受生命中的点滴美好；同时也激励我们在面对困难时不退缩，勇往直前。因为只有经历了风雨洗礼后的彩虹，才会显得格外绚丽多彩。<|endoftext|>'

### 1.2 处理文本数据

最后，为了将该文本变成我们的第一个训练数据集，我们需要构建对它进行处理，以实现对模型的训练：

- 第一步，构建输入输出组合，所谓输入输出组合就是根据前面的所有字来预测下一个字

为了说明这个问题，我们使用原始文档进行说明

In [147]:
import re
preprocessed = [token for token in re.split(pattern, raw_text) if token.strip()] #将原始文档进行分割去除空字符串

for i in range(1, len(preprocessed)):
    input = preprocessed[:i]
    output = preprocessed[i]

    print("".join(input), "——>", output)


每 ——> 一
每一 ——> 滴
每一滴 ——> 汗
每一滴汗 ——> 水
每一滴汗水 ——> 都
每一滴汗水都 ——> 是
每一滴汗水都是 ——> 未
每一滴汗水都是未 ——> 来
每一滴汗水都是未来 ——> 花
每一滴汗水都是未来花 ——> 朵
每一滴汗水都是未来花朵 ——> 的
每一滴汗水都是未来花朵的 ——> 养
每一滴汗水都是未来花朵的养 ——> 分
每一滴汗水都是未来花朵的养分 ——> —
每一滴汗水都是未来花朵的养分— ——> —
每一滴汗水都是未来花朵的养分—— ——> 在
每一滴汗水都是未来花朵的养分——在 ——> 生
每一滴汗水都是未来花朵的养分——在生 ——> 命
每一滴汗水都是未来花朵的养分——在生命 ——> 的
每一滴汗水都是未来花朵的养分——在生命的 ——> 旅
每一滴汗水都是未来花朵的养分——在生命的旅 ——> 途
每一滴汗水都是未来花朵的养分——在生命的旅途 ——> 上
每一滴汗水都是未来花朵的养分——在生命的旅途上 ——> ，
每一滴汗水都是未来花朵的养分——在生命的旅途上， ——> 我
每一滴汗水都是未来花朵的养分——在生命的旅途上，我 ——> 们
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们 ——> 每
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每 ——> 个
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个 ——> 人
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人 ——> 都
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都 ——> 是
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是 ——> 自
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自 ——> 己
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己 ——> 故
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己故 ——> 事
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己故事 ——> 中
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己故事中 ——> 的
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己故事中的 ——> 主
每一滴汗水都是未来花朵的养分——在生命的旅途上，我们每个人都是自己故事中的主

实际上，输入和输出都是固定的长度，例如输入选择前面4个字，输出是输入向后移动1个字，这样有利于计算机处理，否则在运算时候对于不固定的长度处理复杂度较高，现在修改一下程序：

In [96]:
context_size = 4
for i in range(0, len(preprocessed) - context_size): # -context_size 是因为最后一组不足以构建一个组合
    # 输入取固定的长度
    input = preprocessed[i:i+context_size]

    # 输出是输入向后移动一个字的固定长度
    output = preprocessed[i+1:i+context_size+1]

    print("".join(input), "——>", "".join(output))


每一滴汗 ——> 一滴汗水
一滴汗水 ——> 滴汗水都
滴汗水都 ——> 汗水都是
汗水都是 ——> 水都是未
水都是未 ——> 都是未来
都是未来 ——> 是未来花
是未来花 ——> 未来花朵
未来花朵 ——> 来花朵的
来花朵的 ——> 花朵的养
花朵的养 ——> 朵的养分
朵的养分 ——> 的养分—
的养分— ——> 养分——
养分—— ——> 分——在
分——在 ——> ——在生
——在生 ——> —在生命
—在生命 ——> 在生命的
在生命的 ——> 生命的旅
生命的旅 ——> 命的旅途
命的旅途 ——> 的旅途上
的旅途上 ——> 旅途上，
旅途上， ——> 途上，我
途上，我 ——> 上，我们
上，我们 ——> ，我们每
，我们每 ——> 我们每个
我们每个 ——> 们每个人
们每个人 ——> 每个人都
每个人都 ——> 个人都是
个人都是 ——> 人都是自
人都是自 ——> 都是自己
都是自己 ——> 是自己故
是自己故 ——> 自己故事
自己故事 ——> 己故事中
己故事中 ——> 故事中的
故事中的 ——> 事中的主
事中的主 ——> 中的主角
中的主角 ——> 的主角。
的主角。 ——> 主角。每
主角。每 ——> 角。每一
角。每一 ——> 。每一天
。每一天 ——> 每一天，
每一天， ——> 一天，我
一天，我 ——> 天，我们
天，我们 ——> ，我们都
，我们都 ——> 我们都面
我们都面 ——> 们都面临
们都面临 ——> 都面临着
都面临着 ——> 面临着选
面临着选 ——> 临着选择
临着选择 ——> 着选择，
着选择， ——> 选择，每
选择，每 ——> 择，每一
择，每一 ——> ，每一个
，每一个 ——> 每一个选
每一个选 ——> 一个选择
一个选择 ——> 个选择都
个选择都 ——> 选择都是
选择都是 ——> 择都是一
择都是一 ——> 都是一条
都是一条 ——> 是一条通
是一条通 ——> 一条通往
一条通往 ——> 条通往未
条通往未 ——> 通往未知
通往未知 ——> 往未知的
往未知的 ——> 未知的道
未知的道 ——> 知的道路
知的道路 ——> 的道路。
的道路。 ——> 道路。而
道路。而 ——> 路。而在
路。而在 ——> 。而在这
。而在这 ——> 而在这无
而在这无 ——> 在这无数
在这无数 —

另外，我们还可以给这个数据加载方法设置一个移动幅度，比如每次移动两个字：

In [146]:
context_size = 4
stride = 2
for i in range(0, len(preprocessed) - context_size, stride): # -context_size 是因为最后一组不足以构建一个组合
    # 输入取固定的长度
    input = preprocessed[i : i + context_size]

    # 输出是输入向后移动一个字的固定长度
    output = preprocessed[i + 1 : i + context_size + 1]

    print("".join(input), "——>", "".join(output))


每一滴汗 ——> 一滴汗水
滴汗水都 ——> 汗水都是
水都是未 ——> 都是未来
是未来花 ——> 未来花朵
来花朵的 ——> 花朵的养
朵的养分 ——> 的养分—
养分—— ——> 分——在
——在生 ——> —在生命
在生命的 ——> 生命的旅
命的旅途 ——> 的旅途上
旅途上， ——> 途上，我
上，我们 ——> ，我们每
我们每个 ——> 们每个人
每个人都 ——> 个人都是
人都是自 ——> 都是自己
是自己故 ——> 自己故事
己故事中 ——> 故事中的
事中的主 ——> 中的主角
的主角。 ——> 主角。每
角。每一 ——> 。每一天
每一天， ——> 一天，我
天，我们 ——> ，我们都
我们都面 ——> 们都面临
都面临着 ——> 面临着选
临着选择 ——> 着选择，
选择，每 ——> 择，每一
，每一个 ——> 每一个选
一个选择 ——> 个选择都
选择都是 ——> 择都是一
都是一条 ——> 是一条通
一条通往 ——> 条通往未
通往未知 ——> 往未知的
未知的道 ——> 知的道路
的道路。 ——> 道路。而
路。而在 ——> 。而在这
而在这无 ——> 在这无数
这无数的 ——> 无数的选
数的选择 ——> 的选择中
选择中， ——> 择中，有
中，有一 ——> ，有一种
有一种选 ——> 一种选择
种选择是 ——> 选择是永
择是永恒 ——> 是永恒不
永恒不变 ——> 恒不变的
不变的— ——> 变的——
的——那 ——> ——那就
—那就是 ——> 那就是付
就是付出 ——> 是付出努
付出努力 ——> 出努力。
努力。每 ——> 力。每一
。每一次 ——> 每一次的
一次的努 ——> 次的努力
的努力， ——> 努力，无
力，无论 ——> ，无论大
无论大小 ——> 论大小，
大小，都 ——> 小，都在
，都在悄 ——> 都在悄然
在悄然改 ——> 悄然改变
然改变着 ——> 改变着我
变着我们 ——> 着我们的
我们的命 ——> 们的命运
的命运轨 ——> 命运轨迹
运轨迹。 ——> 轨迹。它
迹。它们 ——> 。它们累
它们累积 ——> 们累积起
累积起来 ——> 积起来，
起来，成 ——> 来，成为
，成为我 ——> 成为我们
为我们成 ——> 我们成长
们成长道 ——> 成长道路
长道路上 ——> 道路上最
路上最宝 —

- 第二步，通过上一节的简单分词器进行 `token` 化

In [101]:
import re
context_size = 4
stride = 2
for i in range(0, len(preprocessed) - context_size, stride): # -context_size 是因为最后一组不足以构建一个组合
    # 输入取固定的长度
    input = preprocessed[i:i+context_size]

    # 输出是输入向后移动一个字的固定长度
    output = preprocessed[i+1:i+context_size+1]

    print(tokenizer.encode("".join(input)), "---->", tokenizer.encode("".join(output)))

[199, 6, 210, 204] ----> [6, 210, 204, 201]
[210, 204, 201, 298] ----> [204, 201, 298, 176]
[201, 298, 176, 184] ----> [298, 176, 184, 188]
[176, 184, 188, 256] ----> [184, 188, 256, 185]
[188, 256, 185, 223] ----> [256, 185, 223, 41]
[185, 223, 41, 45] ----> [223, 41, 45, 1]
[41, 45, 1, 1] ----> [45, 1, 1, 87]
[1, 1, 87, 219] ----> [1, 87, 219, 80]
[87, 219, 80, 223] ----> [219, 80, 223, 170]
[80, 223, 170, 293] ----> [223, 170, 293, 7]
[170, 293, 7, 319] ----> [293, 7, 319, 145]
[7, 319, 145, 32] ----> [319, 145, 32, 199]
[145, 32, 199, 12] ----> [32, 199, 12, 25]
[199, 12, 25, 298] ----> [12, 25, 298, 176]
[25, 298, 176, 255] ----> [298, 176, 255, 121]
[176, 255, 121, 165] ----> [255, 121, 165, 22]
[121, 165, 22, 13] ----> [165, 22, 13, 223]
[22, 13, 223, 16] ----> [13, 223, 16, 262]
[223, 16, 262, 5] ----> [16, 262, 5, 199]
[262, 5, 199, 6] ----> [5, 199, 6, 99]
[199, 6, 99, 319] ----> [6, 99, 319, 145]
[99, 319, 145, 32] ----> [319, 145, 32, 298]
[145, 32, 298, 313] ----> [32, 298

- 最后，我们创建一个 `GPTDataset` 来实现数据加载，从输入文本中提取块

In [134]:
import torch
from torch.utils.data import Dataset, DataLoader
class GPTDataset(Dataset):
    def __init__(self, text, tokenizer, context_size, stride):
        self.input_ids = []
        self.output_ids = []

        token_ids = tokenizer.encode(text)

        for i in range(0, len(token_ids) - context_size, stride): # -context_size 是因为最后一组不足以构建一个组合
            # 输入取固定的长度
            input = token_ids[i:i+context_size]

            # 输出是输入向后移动一个字的固定长度
            output = token_ids[i+1:i+context_size+1]

            self.input_ids.append(torch.tensor(input))
            self.output_ids.append(torch.tensor(output))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.output_ids[index]


使用 `create_dataloader` 来创建一个数据加载器，注意这里有个参数 `batch_size`，指的是批量大小，对数据集进行批量并行计算可提高效率，上面我们输出的那些例子都是 `batch_size=1` 的情况，实际训练中，这个参数可进行调整以提升效率，使结果更加稳定。

In [135]:
def create_dataloader(txt, vocab, batch_size=4, max_length=16, 
                         stride=8, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = SimpleTokenizer(vocab)

    # Create dataset
    dataset = GPTDataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

使用 `create_vocab` 方法可临时获得词汇表

In [142]:
def create_vocab(raw_text):
    pattern = r'([\u4e00-\u9fff，。_！？、；：“”‘’()（）——])' # 匹配常见的中文标点符号
    preprocessed = [token for token in re.split(pattern, raw_text) if token.strip()] # 分割并移除空字符串

    all_tokens = sorted(set(preprocessed))
    all_tokens.extend(["<|endoftext|>", "<|unk|>"])
    vocab = {token:integer for integer,token in enumerate(all_tokens)}

    return vocab

- 让我们针对上下文大小为 4 的 LLM 测试批量大小为 1 的数据加载器：

In [143]:
with open("the-road.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

vocab = create_vocab(raw_text)
context_size = 4
stride = 1
batch_size = 1
dataloader = create_dataloader(
    raw_text, vocab=vocab, batch_size=batch_size, max_length=context_size, stride=stride, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[199,   6, 210, 204]]), tensor([[  6, 210, 204, 201]])]


In [144]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[  6, 210, 204, 201]]), tensor([[210, 204, 201, 298]])]


- 我们还可以创建分批输出
- 请注意，我们在这里增加了步幅，这样批次之间就不会出现重叠，因为更多的重叠可能会导致过度拟合

In [145]:
dataloader = create_dataloader(raw_text, vocab=vocab, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("输入:\n", inputs)
print("\n输出:\n", targets)

输入:
 tensor([[199,   6, 210, 204],
        [201, 298, 176, 184],
        [188, 256, 185, 223],
        [ 41,  45,   1,   1],
        [ 87, 219,  80, 223],
        [170, 293,   7, 319],
        [145,  32, 199,  12],
        [ 25, 298, 176, 255]])

输出:
 tensor([[  6, 210, 204, 201],
        [298, 176, 184, 188],
        [256, 185, 223,  41],
        [ 45,   1,   1,  87],
        [219,  80, 223, 170],
        [293,   7, 319, 145],
        [ 32, 199,  12,  25],
        [298, 176, 255, 121]])


好了，到此数据准备完成，下一步我们进入模型训练

## 2. 模型架构与实现

## 3. 模型训练与评估

## 4. 文本生成与微调

## 结语